In [1]:
# Installing dependencies
import numpy as np
import pandas as pd
from dLoader import DataLoader, BuySell

In [2]:
def softmax(x):
    # Calculate softmax of x
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [3]:
class MACross:
    # Moving Average Fast Slow Crossover Strategy
    # Check Fast Moving Average against Slow Moving Average
    # Also checking Current Volume is above 
    # Volume Moving Average
    def __init__(self, fast=3, slow=6):
        self.fast=3
        self.slow=6
    
    def check_ma_above(self, df):
        # Create Moving Averages for Price and Volume
        ndf = self.get_ma(df)
        # Check if price and volume is above Moving Averages
        if ndf['MA-Above'].iloc[-1] and ndf['MA-Vol-Above'].iloc[-1]:
            return True
        return False

    def get_ma(self, df):
        # MA Column Creations
        df['Fast'] = df['Close'].rolling(self.fast).mean()
        df['Slow'] = df['Low'].rolling(self.slow).mean()
        df['MA-Vol'] = df['Volume'].rolling(self.slow).mean()
        df['MA-Above'] = df['Fast'] > df['Slow']
        df['MA-Vol-Above'] = df['Volume'] > df['MA-Vol']
        df['HLT'] = (df['High'] / df['Low'].shift(1) - 1).shift(-1)
        df['OLN'] = df['Open'] / df['Low'] - 1
        return df.dropna()


In [4]:
class GapTable:
    # Using data to create a Gap Percentage Table
    # Table contain Counts, Percentage, and Softmax
    def __init__(self):
        pass
    
    def table_with_reduction(self, data, step=0.01, n=10):
        # Create reduction dictionary base on High Low percentage difference
        # Moving Average Crossover and Volume over Moving Average Volume
        # Extract Columns from DataFrame
        hl = data['HLT']
        mask_dic = {'MA': data['MA-Above'], 
                    'V': data['MA-Vol-Above']}
        # Dictionary to DataFrame from Percentage Range
        df = self.check_isin_pairs(hl, mask_dic, self.create_range(hl, step=step))
        # Replacing counts less or equal to n to zero
        cols = ['Count', 'MA-Count', 'V-Count']
        df.loc[:, cols] = df.loc[:, cols].where(df >= n, 0)
        # Get mask of values that is not 0
        mask = (df.loc[:, cols] != 0).sum(1) > 0
        # Apply the mask and reindex the DataFrame copy
        df = df[mask].reset_index(drop=True).copy()
        # Calculate Softmax from sum of [Count, MA-Count, V-Count]
        pct = df.loc[:, cols].sum(1) / df.loc[:, cols].sum(1).sum()
        df['Softmax'] = softmax(pct)
        # Return the table with only the softmax probability of the reduction percentage Range
        return df.drop(columns=cols)

    def create_table(self, 
                     data=None, 
                     step=None, 
                     a_col=None, a_shift=None, 
                     b_col=None, b_shift=None):
        a = self.get_col(data=data, col=a_col, s=a_shift)
        b = self.get_col(data=data, col=b_col, s=b_shift)
        # Create Probability table
        pct =self.get_pct(a, b)
        pairs = self.create_range(pct, step)
        return self.create_probability_df(pct, pairs)
    
    def create_probability_df(self, pct, pairs):
        # Create Probability Table base on Gap percentage range
        dic = {}
        for i, pair in enumerate(pairs):
            # Check if Gap percentage is within range
            isin = (pct >= pair[0]) & (pct < pair[1])
            # Assign the total counts into dictionary
            dic[i] = {'A': pair[0], 
                      'B': pair[1],
                      'Counts': len(pct[isin])}
        # Turn dictionary into DataFrame
        df = pd.DataFrame.from_dict(dic, orient='index')
        # Calculate Percentage
        df['PCT'] = df['Counts'] / df['Counts'].sum()
        # Calculate Softmax
        df['Softmax'] = softmax(df['PCT'])
        return df
    
    def check_isin_pairs(self, array, mask_dic, pairs):
        dic = {}
        # Loop through pairs and get counts for percentage range
        for i, pair in enumerate(pairs):
            arr = np.array(array)
            dic[i] = {'A': pair[0], 
                      'B': pair[1], 
                      'Count': self.check_isin(arr, pair)}
            # Loop through mask_dic and update dictionary
            for k in mask_dic.keys():
                dic[i].update({k+'-Count': self.check_isin(arr[mask_dic[k]], pair)})
        # Return dictionay into Pandas DataFrame
        return pd.DataFrame.from_dict(dic, orient='index')
    
    def check_isin(self, array, pair):
        # Check values within the range
        mask = (array >= pair[0]) & (array < pair[1])
        # Return the count
        return mask.sum()

    def create_range(self, pct, step):
        # Getting Gap percentage range
        total = pct.max() - pct.min()
        drange = np.arange(.0, np.round(total, 2), step) + np.round(pct.min(), 2)
        return np.stack([drange[:-1], drange[1:]]).T
    
    def get_col(self, col, data, s=None):
        # Change different between data columns
        if s is not None:
            return data[col].shift(s)
        return data[col]

    def get_pct(self, a, b):
        # Percentage difference between a and b
        return a / b  - 1

In [5]:
# Load data
stock_data = DataLoader('AAPL')
# Get a range of Past Data to create a GapTable
pdata = stock_data.get_data('2017-01-01', '2017-12-31')

# Assigning Data Manipulation Class
mc = MACross()
gt = GapTable()
pdata = mc.get_ma(pdata)

In [6]:
pTable = gt.table_with_reduction(pdata, step=.005, n=10)
pTable

,A,B,Softmax
0,0.000,0.005,0.151944
1,0.005,0.010,0.186473
2,0.010,0.015,0.179560
3,0.015,0.020,0.171876
4,0.020,0.025,0.160005
5,0.025,0.030,0.150142
